# Modelo de cruce de calles

This notebook presents a tutorial for beginners on how to create a simple agent-based model with the [agentpy](https://agentpy.readthedocs.io) package. 
It demonstrates how to create a basic model with a custom agent type, run a simulation, record data, and visualize results.

In [1]:
# Model design
import agentpy as ap
import numpy as np 
import random
# Visualization
import seaborn as sns
import pandas as pd

## About the model

The model explores the distribution of wealth under a trading population of agents. 
Each agent starts with one unit of wealth. 
During each time-step, each agents with positive wealth 
randomly selects a trading partner and gives them one unit of their wealth.
We will see that this random interaction will create an inequality of wealth that 
follows a [Boltzmann distribution](http://www.phys.ufl.edu/~meisel/Boltzmann.pdf).
The original version of this model been written in [MESA](https://mesa.readthedocs.io/) 
and can be found [here](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html).

## Model definition

Se declara el agente carro

In [314]:
val = ["arriba","izquierda","abajo","derecha"]
class CarAgent(ap.Agent):
    def setup(self):
        self.group = val[random.randint(0,3)]
        if(self.group=="arriba"):
            self.pos_X = 45
            self.pos_Y = 95
            self.speed_X = 0
            self.speed_Y = -random.randint(30,50)
        elif(self.group=="izquierda"):
            self.pos_X = 5
            self.pos_Y = 45
            self.speed_X = random.randint(30,50)
            self.speed_Y = 0
        elif(self.group=="abajo"):
            self.pos_X = 55
            self.pos_Y = 5
            self.speed_X = 0
            self.speed_Y = random.randint(30,50)
        elif(self.group=="derecha"):
            self.pos_X = 95
            self.pos_Y = 55
            self.speed_X = -random.randint(30,50)
            self.speed_Y = 0
    def move(self):
        res = False
        if(self.group=="arriba"):
            res = self.model.semaforos[0].on
        elif(self.group=="izquierda"):
            res = self.model.semaforos[1].on
        elif(self.group=="abajo"):
            res = self.model.semaforos[2].on
        elif(self.group=="derecha"):
            res = self.model.semaforos[3].on
        return res
    def drive(self):
        if CarAgent.move(self):
            self.pos_X += self.speed_X*0.1
            self.pos_Y += self.speed_Y*0.1

In [315]:
class SemaforoAgent(ap.Agent):
    def setup(self):
        self.on = False
    def cambia(self):
        self.on = True

Esto es lo que se hace en el modelo:
- `setup` crea el arreglo de agentes
- `step` metodos que se llaman en cada iteracion
- `update` datos que se guardan en cada iteracion
- `end` llamado al finalizar el modelo

In [316]:
class StreetModel(ap.Model):

    def setup(self):
        self.cars = ap.AgentList(self, self.p.agents, CarAgent)
        self.semaforos = ap.AgentList(self, 4, SemaforoAgent)
        self.actual = 0
        self.semaforos[self.actual].on = True
        
    def step(self):
        self.cars.drive()

        if self.t % self.p.cambia == 0:
            self.semaforos[self.actual].on = False
            self.actual += 1
            if(self.actual>3):
                self.actual = 0
            self.semaforos[self.actual].on = True

    def update(self):
        self.cars.record('pos_X')
        self.cars.record('pos_Y')
        self.semaforos.record('on')

    def end(self):
        self.cars.record('pos_X')
        self.cars.record('pos_Y')

## Simulation run

Parametros de una simulacion

In [317]:
parameters = {
    'agents': 5,
    'steps': 100,
    'seed': 42,
    'cambia': 30
}

Ejecucion de una simulacion

In [318]:
model = StreetModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.043883
Simulation finished


## Output analysis

DataDict de la simulacion

In [319]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 4 keys
'variables': 
    'CarAgent': DataFrame with 2 variables and 505 rows
    'SemaforoAgent': DataFrame with 1 variable and 404 rows
'reporters': DataFrame with 1 variable and 1 row
}

Informacion de la simulacion

In [320]:
results.info

{'model_type': 'StreetModel',
 'time_stamp': '2021-11-22 20:23:59',
 'agentpy_version': '0.1.4',
 'python_version': '3.9.0',
 'experiment': False,
 'completed': True,
 'created_objects': 9,
 'completed_steps': 100,
 'run_time': '0:00:00.043883'}

Tabla de valores de encendido para cada semaforo

In [321]:
results.variables.SemaforoAgent

on
obj_id t        
6      0    True
       1    True
       2    True
       3    True
       4    True
...          ...
9      96   True
       97   True
       98   True
       99   True
       100  True

[404 rows x 1 columns]

In [322]:
escrito = pd.ExcelWriter('semaforos.xlsx')
# escribir el DataFrame en excel
results.variables.SemaforoAgent.to_excel(escrito)
# guardar el excel
escrito.save()

Tabla de posiciones para cada carro

In [323]:
results.variables.CarAgent

pos_X  pos_Y
obj_id t                
1      0     55.0    5.0
       1     55.0    5.0
       2     55.0    5.0
       3     55.0    5.0
       4     55.0    5.0
...           ...    ...
5      96    55.0  140.0
       97    55.0  140.0
       98    55.0  140.0
       99    55.0  140.0
       100   55.0  140.0

[505 rows x 2 columns]

In [324]:
escrito = pd.ExcelWriter('carros.xlsx')
# escribir el DataFrame en excel
results.variables.CarAgent.to_excel(escrito)
# guardar el excel
escrito.save()